In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

model = models.resnet50()
num_features = model.fc.in_features

# 새로운 linear layer로 대체
model.fc = nn.Linear(num_features, 101)

In [ ]:
state_path = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/resnet50model.pth'
# https://github.com/Herick-Asmani/Food-101-classification-using-ResNet-50의 food101 모델을 초기모델로 활용
checkpoint_state = torch.load(state_path)
model.load_state_dict(checkpoint_state['model_state'])

<All keys matched successfully>

In [ ]:
imgfolderpath = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/images/train/foods'

In [ ]:
import os
labels = []
img_path_list = []
for label in os.listdir(imgfolderpath):
    labels.append(label)
    label_path = os.path.join(imgfolderpath, label)
    if os.path.isdir(label_path):
        if os.path.isdir(label_path):
            img_files = os.listdir(label_path)
            img_paths = []
            for img_file in img_files:
                if img_file.endswith('.jpg'):
                    img_paths.append(os.path.join(label_path, img_file))
            img_path_list.extend(img_paths)

print(len(img_path_list))

5297


In [ ]:
from PIL import UnidentifiedImageError

for i, img_path in enumerate(img_path_list):
    if i % 10 == 0:
        print(f"{i}/{len(img_path_list)}")
    try:
        image = Image.open(img_path)
        pass
    except UnidentifiedImageError:
        os.remove(img_path)
        continue

In [ ]:
labels

['Orange Juice',
 'Milk',
 'Salad',
 'Egg',
 'Candied Sweet Potatoes',
 'Kimchi',
 'Rice',
 'Bean Sprouts',
 'Chicken',
 'Bibimbap',
 'Pork Cutlet']

In [ ]:
num_classes = len(labels)

In [ ]:
# 기존 모델 파라미터 동결
for param in model.parameters():
    param.requies_grad = False

# 모델 아랫단에 분류기 추가
model = nn.Sequential(model,
                      nn.Linear(model.fc.out_features, num_classes))

In [ ]:
print(model)

In [ ]:
labels.index('Bibimbap')

9

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        image_path = self.data_list[index]
        folder_name = os.path.basename(os.path.dirname(image_path))
        label = labels.index(folder_name)
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
from torchvision.transforms import transforms

# https://github.com/Herick-Asmani/Food-101-classification-using-ResNet-50
transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(45),
                                       transforms.RandomAffine(45),
                                       transforms.ColorJitter(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# 데이터셋 크기
train_dataset = CustomDataset(img_path_list, transform)
dataset_size = len(train_dataset)

# 인덱스를 생성하여 train과 valid 데이터셋을 나눔
indices = list(range(dataset_size))
train_indices, valid_indices = train_test_split(indices, test_size=0.2)

# 데이터셋과 인덱스를 사용하여 train_loader와 valid_loader를 생성
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_indices)

batch_size = 64  # 배치 크기


train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
valid_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=valid_sampler)

In [ ]:
import torch.optim as optim

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5)

In [ ]:
def calc_accuracy(prediction, label):
    predicted_classes = torch.argmax(prediction, dim=1)

    # 정확한 예측 개수 계산
    correct_predictions = (predicted_classes == label).sum().item()

    # 전체 데이터 개수
    total_samples = batch.size(0)

    # 정확도 계산
    accuracy = correct_predictions / total_samples

    return accuracy


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model.to(device)

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/BAE-APA/model/checkpoint.pth'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
optimizer.param_groups[0]['lr'] = checkpoint['learning_rate']

In [ ]:
import time
epochs = 100

start = time.time()
best_accuracy = 0.0
for epoch in range(epochs):

    train_accuracy_list, valid_accuracy_list = [], []
    train_loss, valid_loss = 0.0, 0.0

    model.train()
    print('Train Start')
    for _iter, (batch, label) in enumerate(train_loader):
        print(f'Iter : ({_iter}/{len(train_loader)})')
        batch = batch.cuda()
        label = label.cuda()

        prediction = model(batch)
        loss = criterion(prediction, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_accuracy_list.append(calc_accuracy(prediction, label))
        train_loss += loss.item()

    train_total_accuracy = sum(train_accuracy_list) / len(train_accuracy_list)

    with torch.no_grad():
        model.eval()

        for batch, label in valid_loader:
            batch = batch.cuda()
            label = label.cuda()

            prediction = model(batch)
            loss = criterion(prediction, label)

            valid_accuracy_list.append(calc_accuracy(prediction, label))
            valid_loss += loss.item()

        valid_accuracy = sum(valid_accuracy_list) / len(valid_accuracy_list)

        if valid_accuracy > best_accuracy:
            best_accuracy = valid_accuracy
            best_model_state = model.state_dict()
            best_optim_state = optimizer.state_dict()
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'learning_rate': optimizer.param_groups[0]['lr']
            }, 'checkpoint.pth')
    print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || [{epoch}/{epochs}], train_loss = {train_total_loss:.4f}, train_accuracy = {train_total_accuracy:.2f}, valid_loss = {valid_total_loss:.4f}, valid_accuracy = {valid_total_accuracy:.2f}""")
    print("learning rate " + str(optimizer.param_groups[0]['lr']))
elapsed = time.time() - start
print(f"Elapsed time : {elapsed} sec")

2023-06-29 12:08:48 || [0/100], train_loss = 3.3137, train_accuracy = 0.99, valid_loss = 1.7403, valid_accuracy = 0.97
learning rate 0.01
2023-06-29 12:11:54 || [1/100], train_loss = 3.8051, train_accuracy = 0.98, valid_loss = 1.8362, valid_accuracy = 0.97
learning rate 0.01
2023-06-29 12:15:05 || [2/100], train_loss = 3.7892, train_accuracy = 0.98, valid_loss = 2.1166, valid_accuracy = 0.96
learning rate 0.01
2023-06-29 12:18:13 || [3/100], train_loss = 4.2227, train_accuracy = 0.98, valid_loss = 1.7652, valid_accuracy = 0.97
learning rate 0.01
2023-06-29 12:21:21 || [4/100], train_loss = 6.1505, train_accuracy = 0.97, valid_loss = 1.4962, valid_accuracy = 0.97
learning rate 0.01
2023-06-29 12:24:32 || [5/100], train_loss = 5.2525, train_accuracy = 0.97, valid_loss = 2.6377, valid_accuracy = 0.95
learning rate 0.01
2023-06-29 12:27:40 || [6/100], train_loss = 5.6814, train_accuracy = 0.98, valid_loss = 2.0749, valid_accuracy = 0.96
learning rate 0.01
2023-06-29 12:30:46 || [7/100], tr

In [ ]:
torch.save({
    'model_state_dict': best_model_state,
    'optimizer_state_dict': best_optim_state,
    'learning_rate': 0.01
}, checkpoint_path)